In [1]:
import pandas as pd

In [2]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.search(emoj, data)

In [3]:
labels_df = pd.read_csv("../relations_labels.csv")#, delim_whitespace=True, error_bad_lines=False, header = None)

print("Starting: ", len(labels_df))


Starting:  964


In [4]:
labels_df.head()

,uri,label
0,<http://www.wikidata.org/entity/P4882>,segmental innervation
1,<http://www.wikidata.org/entity/P1339>,number injured
2,<http://www.wikidata.org/entity/P1339>,injury toll
3,<http://www.wikidata.org/entity/P1339>,number of injured
4,<http://www.wikidata.org/entity/P1339>,injured


In [5]:
import elasticsearch
config = {
    'host': 'localhost'
}
es = elasticsearch.Elasticsearch([config,], timeout=300)

In [6]:
if es.indices.exists("wikidata_bio_subset_3_relation_updated_index"):
    es.indices.delete(index="wikidata_bio_subset_3_relation_updated_index")

/tmp/ipykernel_301685/3631762966.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists("wikidata_bio_subset_3_relation_updated_index"):


In [7]:
es.indices.create(index="wikidata_bio_subset_3_relation_updated_index")

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'wikidata_bio_subset_3_relation_updated_index'}

In [8]:
from elasticsearch import helpers
import tqdm.notebook as tq

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in tq.tqdm(df_iter):
#         print(document.to_dict())
        yield {
                "_index": 'wikidata_bio_subset_3_relation_updated_index',
                "_type": "_doc",
                "_id" : f"{index+1}",
                "_source": document.to_dict(),
            }
    print("Complete")
    #raise StopIteration
elasticsearch.helpers.bulk(es, doc_generator(labels_df))

0it [00:00, ?it/s]

Complete


/home/krishanu/anaconda3/envs/pytorch/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(964, [])

In [9]:
indexName="wikidata_bio_subset_3_relation_updated_index"
elasticResults=es.search(index=indexName, body={
        "from": 0,
        "size": 100,
        "query": {
            "match": {
                "label":"inheritance"
            }
        }
    })

print(elasticResults)

/tmp/ipykernel_301685/2536444380.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  elasticResults=es.search(index=indexName, body={


{'took': 689, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 5.797125, 'hits': [{'_index': 'wikidata_bio_subset_3_relation_updated_index', '_type': '_doc', '_id': '72', '_score': 5.797125, '_source': {'uri': '<http://www.wikidata.org/entity/P1199>', 'label': 'mode of inheritance'}}]}}
